In [9]:
%matplotlib inline
import numpy as np
import compas
from compas.datastructures import Mesh
import compas.datastructures as ds
from compas_plotters import Plotter
import matplotlib.pyplot as plt
import networkx as nx 
import json
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

In [10]:
fin = open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDSS_test/data/my_igml_703.json')
j = json.loads(fin.read())

fin_2 = open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDSS_test/data/out.json')
j_2 = json.loads(fin_2.read())
fin_3 = open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDSS_test/data/my_out.json')
j_3 = json.loads(fin_3.read())

fin_4 = open("C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/hugo_test/Newtest_output.json")
j_4 = json.loads(fin_4.read())

In [11]:
def are_cells_adjacent(a, b, j):
    if a not in j['PrimalSpaceFeatures']['CellSpace']:
        return False
    if b not in j['PrimalSpaceFeatures']['CellSpace']:
        return False
    ca = j['PrimalSpaceFeatures']['CellSpace'][a]
    cb = j['PrimalSpaceFeatures']['CellSpace'][b]
    # find the dual node
    dual_ca = j['SpaceLayers'][ca['duality-spacelayer']][ca['duality']]
    dual_cb = j['SpaceLayers'][cb['duality-spacelayer']][cb['duality']]
    bFound = False
    # iterate over the edges to see whether b is in a
    for e in dual_ca['edges']:
        if (e['destination'] == cb['duality']):
            bFound = True
            break
    if (bFound == False):
        # iterate over the edges to see whether a is in b
        for e in dual_cb['edges']:
            if (e['destination'] == ca['duality']):
                bFound = True
                break
    return bFound

out = are_cells_adjacent('C1','C2',j_3)
print(out)

False


In [12]:

import os
import sys
import json

fin = open('C:/Users/SID-DRW/OneDrive - Delft University of Technology/Documents/GitHub/HDSS_test/data/my_igml_703.json')

def main():
    j = json.loads(fin.read())

    print ("IndoorJSON version:", j['version'])
    print ("Number CellSpace:", len(j['PrimalSpaceFeatures']))
    print ("Number dual graphs (SpaceLayers):", len(j['SpaceLayers']))

    #-- are Cell C1 and C8 adjacent? [either edge C1->C8 or C8->C1]
    print ("C1 and C2 adjacent?", are_cells_adjacent('C1', 'C2', j))
    #-- are Cell C1 and C8 adjacent? 
    print ("C1 and C3 adjacent?", are_cells_adjacent('C1', 'C3', j))
    #-- are Cell C1 and C9 adjacent? 
    print ("C1 and C9 adjacent?", are_cells_adjacent('C1', 'C9', j))


    #-- what is the dual of Node R10?
    dual_r10 = None
    for graphid in j['SpaceLayers']:
        if ('R10' in j['SpaceLayers'][graphid]):
            dual_r10 = j['SpaceLayers'][graphid]['R10']['duality']
    print ('Dual of R10:', str(dual_r10))


def are_cells_adjacent(a, b, j):
    if a not in j['PrimalSpaceFeatures']:
        return False
    if b not in j['PrimalSpaceFeatures']:
        return False
    ca = j['PrimalSpaceFeatures'][a]
    cb = j['PrimalSpaceFeatures'][b]
    # find the dual node
    dual_ca = j['SpaceLayers'][ca['duality-spacelayer']][ca['duality']]
    dual_cb = j['SpaceLayers'][cb['duality-spacelayer']][cb['duality']]
    bFound = False
    # iterate over the edges to see whether b is in a
    for e in dual_ca['edges']:
        if (e['destination'] == cb['duality']):
            bFound = True
            break
    if (bFound == False):
        # iterate over the edges to see whether a is in b
        for e in dual_cb['edges']:
            if (e['destination'] == ca['duality']):
                bFound = True
                break
    return bFound


if __name__ == '__main__':
    main()

IndoorJSON version: 0.2
Number CellSpace: 2
Number dual graphs (SpaceLayers): 1
C1 and C2 adjacent? False
C1 and C3 adjacent? False
C1 and C9 adjacent? False
Dual of R10: None


In [13]:
indoor_json = {
    "SpaceLayers": {
        "SLs_40": {
            "N1": {
                "type": "Node",
                "duality": "C1",
                "geometry": {"type": "Point", "boundaries": 20},
                "edges": [{"type": "Edge", "destination": "SD1", "weight": 1.0, "extra_nodes": None}]
            },
            "N2": {
                "type": "Node",
                "duality": "C2",
                "geometry": {"type": "Point", "boundaries": 25},
                "edges": []
            },
            "SD1": {
                "type": "Node",
                "duality": "B1",
                "geometry": {"type": "Point", "boundaries": 7},
                "edges": [{"type": "Edge", "destination": "N2", "weight": 1.0, "extra_nodes": None}]
            }
        }
    }
}

# Extracting edges from the JSON structure
edges = []
node_keys = list(indoor_json["SpaceLayers"]["SLs_40"].keys())
for node_key, node_value in indoor_json["SpaceLayers"]["SLs_40"].items():
    for edge in node_value.get("edges", []):
        source_index = node_keys.index(node_key)
        destination_index = node_keys.index(edge["destination"])
        edges.append([source_index, destination_index])

print(edges)


[[0, 2], [2, 1]]


In [14]:
'''def get_edges(j):

    for sl in j['SpaceLayers']:
        edges = []
        node_keys = list(j["SpaceLayers"][sl].keys())
        for node_key, node_value in j["SpaceLayers"][sl].items():
            for edge in node_value.get("edges", []):
                source_index = node_keys.index(node_key)
                destination_index = node_keys.index(edge["destination"])
                edges.append([source_index, destination_index])

    return edges'''

def get_edges(j):
    for sl in j['SpaceLayers']:

        edges_modified = []
        for node_key, node_value in j["SpaceLayers"][sl].items():
            source_boundary = node_value["geometry"]["boundaries"]
            for edge in node_value.get("edges", []):
                destination_node = j["SpaceLayers"][sl][edge["destination"]]
                destination_boundary = destination_node["geometry"]["boundaries"]
                edges_modified.append([source_boundary, destination_boundary])

    return edges_modified

print(get_edges(j))
print(get_edges(j_4))
print(get_edges(j_2))
print(len(get_edges(j_2)))


[[0, 1]]
[[0, 2], [2, 1]]
[[0, 2], [2, 0], [2, 8], [2, 10], [2, 12], [2, 14], [2, 16], [8, 2], [10, 2], [12, 2], [14, 2], [16, 2], [16, 23], [16, 23], [16, 21], [16, 27], [16, 32], [21, 16], [23, 16], [23, 16], [23, 27], [27, 16], [27, 23], [30, 32], [32, 16], [32, 30], [32, 36], [32, 38], [36, 32], [38, 32]]
30


In [15]:
# Modifying the code to output edges based on the "boundaries" key values instead of node indices

edges_modified = []
for node_key, node_value in indoor_json["SpaceLayers"]["SLs_40"].items():
    source_boundary = node_value["geometry"]["boundaries"]
    for edge in node_value.get("edges", []):
        destination_node = indoor_json["SpaceLayers"]["SLs_40"][edge["destination"]]
        destination_boundary = destination_node["geometry"]["boundaries"]
        edges_modified.append([source_boundary, destination_boundary])

edges_modified


[[20, 7], [7, 25]]